In [1]:
!pip3 install opencv-python ultralytics supervision torchvision torchaudio torch


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
# imports
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import supervision as sv
from collections import defaultdict


/Users/longtruong/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/longtruong/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Loading + Preprocessing

In [3]:
# get frames for initializing 
cap = cv.VideoCapture(0)
model = YOLO('yolov8n.pt')
posemodel = YOLO('yolov8n-pose.pt')
names = model.names
savedFrames = []

itr = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    results = model(frame)
    results2 = posemodel(frame)
    for result in results:
        save_buffer = []
        box_buffer = []
        boxes = result.boxes.cpu().numpy()
        
        for box, xywh in zip(boxes.cls, boxes.xyxy):
            if names[box] == 'person':
                x1, y1, x2, y2 = map(int,xywh)
                
                # print(x, y, w, h)
                person_roi = frame[y1:y2, x1:x2]
                save_buffer.append(person_roi)
        if len(save_buffer) == 1:
            savedFrames.append(save_buffer[0])
        else:
            break
    detections = sv.Detections(
                    xyxy=results[0].boxes.xyxy.cpu().numpy(),
                    confidence=results[0].boxes.conf.cpu().numpy(),
                    class_id=results[0].boxes.cls.cpu().numpy().astype(int),
                    )

    frame = sv.BoxAnnotator(sv.ColorPalette.default(), thickness=3, text_thickness=3, text_scale=1.5).annotate(scene=frame, detections=detections)
    cv.imshow('Live Video', frame)
    if cv.waitKey(1) & 0xFF == 27:  # Press 'Esc' key to exit the loop
        break

cap.release()
cv.destroyAllWindows()


0: 384x640 (no detections), 49.7ms
Speed: 2.8ms preprocess, 49.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 52.2ms
Speed: 2.2ms preprocess, 52.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 49.6ms
Speed: 1.8ms preprocess, 49.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 51.0ms
Speed: 2.0ms preprocess, 51.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 46.6ms
Speed: 1.9ms preprocess, 46.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 59.4ms
Speed: 1.7ms preprocess, 59.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 46.3ms
Speed: 1.9ms preprocess, 46.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 49.5ms
Speed: 1.8ms preprocess, 49.5ms i

In [4]:
tmp = "McLovin_"
itr = 0
for frame in savedFrames:
    bgr = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    cv.imwrite("img/rgb_"+tmp + str(itr) + ".png", frame)
    cv.imwrite("img/bgr_"+tmp + str(itr) + ".png", bgr)
    cv.imwrite("img/hsv_"+tmp + str(itr) + ".png", hsv)
    # print(frame)
    itr += 1

## Model Testing

In [21]:
cap = cv.VideoCapture(0)
model = YOLO('models/segment-hpo/bestMcChimkns.pt')
model2 = YOLO('yolov8n-pose.pt')
viewVal = -1

while True:
    ret, frame = cap.read()

    if not ret:
        break
    if viewVal == -1:
        results = model.track(frame, show=True)
    else:
        results = model2.predict(frame)
    
    annotated_frame = results[0].plot()
    
    if cv.waitKey(1) & 0xFF == 32:
        viewVal *= -1

    cv.imshow('Live Video',annotated_frame)
    if cv.waitKey(1) & 0xFF == 27:  # Press 'Esc' key to exit the loop
        break

cap.release()
cv.destroyAllWindows()


0: 384x640 2 McLovins, 63.2ms
Speed: 2.1ms preprocess, 63.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 McLovins, 56.3ms
Speed: 1.9ms preprocess, 56.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 McLovins, 56.0ms
Speed: 1.8ms preprocess, 56.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 McLovins, 56.8ms
Speed: 2.4ms preprocess, 56.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 McLovins, 54.7ms
Speed: 2.0ms preprocess, 54.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 McLovins, 57.3ms
Speed: 1.6ms preprocess, 57.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 McLovins, 53.9ms
Speed: 1.9ms preprocess, 53.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 McLovins, 54.2ms
Speed: 1.8ms preprocess, 54.2ms inference, 1.3ms postprocess per image at

error: OpenCV(4.8.1) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/video/src/lkpyramid.cpp:1260: error: (-215:Assertion failed) (npoints = prevPtsMat.checkVector(2, CV_32F, true)) >= 0 in function 'calc'


In [9]:
cap = cv.VideoCapture(0)
model = YOLO('yolov8n-pose.pt')


while True:
    ret, frame = cap.read()

    if not ret:
        break
    results = model(frame)
    annotated_frame = results[0].plot()

    cv.imshow('Live Video',annotated_frame)
    if cv.waitKey(1) & 0xFF == 27:  # Press 'Esc' key to exit the loop
        break



0: 384x640 (no detections), 52.9ms
Speed: 2.3ms preprocess, 52.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 49.4ms
Speed: 2.1ms preprocess, 49.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 80.1ms
Speed: 2.4ms preprocess, 80.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 51.2ms
Speed: 1.9ms preprocess, 51.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 48.7ms
Speed: 1.5ms preprocess, 48.7ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 50.3ms
Speed: 1.6ms preprocess, 50.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 54.3ms
Speed: 1.7ms preprocess, 54.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 48.1ms
Speed: 1.6ms preprocess, 48.1ms i

## Frame Sampling

In [13]:
cap = cv.VideoCapture(1)
cacheFrames = []

while True:
    ret, frame = cap.read()

    if not ret:
        break
    
    # results = model(frame)
    # annotated_frame = results[0].plot()
    cacheFrames.append(frame)
    cv.imshow('Live Video',frame)
    if cv.waitKey(1) & 0xFF == 27:  # Press 'Esc' key to exit the loop
        break

cap.release()
cv.destroyAllWindows()

KeyboardInterrupt: 

In [14]:
for frame in cacheFrames:
    
    cv.imshow('Live Video',frame)
    if cv.waitKey(1) & 0xFF == 27:  # Press 'Esc' key to exit the loop
        break

cap.release()
cv.destroyAllWindows()